# find 3 closest poi in a range of 25 mt

In [2]:
import numpy as np

In [7]:
from geopy.distance import vincenty
print(np.sqrt(vincenty((43.701944,7.268333), (43.665278,7.215)).km))

2.43383657301


In [12]:
a = vincenty((43.701944,7.268333), (43.665278,7.215))
a.m

5923.560464100794

In [26]:
pois = ['1','2','3','4']
weights = ['8','2','1','8']
lists = zip(pois,weights)
groups =[':'.join(x) for x in lists]
name = '_'.join(groups)

In [27]:
name

'1:8_2:2_3:1_4:8'

In [225]:
import copy
def near_pois(original_pois, point):
    def add_distance_wrapper(point):
        def add_distance(target):
            point['distance'] = vincenty(target, (float(point['lat']),float(point['long']))).mt
            if point['distance'] < 5:
                point['distance'] = 5
            point['distance_sqrt'] = np.sqrt(point['distance'])
            return point
        return add_distance_wrapper

    MIN_RADIUS = 50
    MIN_RADIUS_SQRT = np.sqrt(MIN_RADIUS)
    
    original_pois = [{'lat':43.701944,'long':7.2683333232,'label':'ciao'}]
    
    new_pois = copy.deepcopy(original_pois)
    
    point = (43.665278,7.215)
    add_distance_function = add_distance_wrapper(point)
    
    near_pois = map(add_distance_function,new_pois)
    near_pois = sorted(lambda x: x['distance'], near_pois)[:3]
    
    filtered_pois = [near_pois.pop(0)]
    filtered_pois += filter(lambda x: x['distance_sqrt'] < MIN_RADIUS_SQRT, near_pois)[:2]

    #weight part
    total_score = sum(1./x['distance_sqrt'] for x in filtered_pois)
    total_weight = 0
    for poi in filtered_pois:
        poi['weight'] = 1./(total_score*poi['distance_sqrt'])
        total_weight += poi['weight']
    for poi in filtered_pois:
        poi['weight'] = int(round(poi['weight'] * 10./ total_weight))
    
    return filtered_pois

# find poi to show on the map

read all

remove bus stations and add position to each 3cixty poi

group by dbpedia uri

foreach group:
    keep entries with the highest score
    remove entries with score < 90

    keep the nearest to the center

In [21]:
import csv
import urllib2
import json
from geopy.distance import vincenty
from SPARQLWrapper import SPARQLWrapper,JSON

NICE_COORDINATES = (43.7034, 7.2663)
THRESHOLD = 85

In [22]:
def add_info(row):
    url = row[0].rsplit('/',1)[1]
    req = urllib2.Request('http://aplicaciones.localidata.com/eldaSuit/place/id/%s?_view=list' % url)
    resp = urllib2.urlopen(req)
    # 200
    body = resp.read()
    result = json.loads(body)
    if len(result['result']['items']) > 0:
        if type(result['result']['items'][0]["geoLocation"]['lat']) == list:
            lat = float(result['result']['items'][0]["geoLocation"]['lat'][0])
        else:
            lat = float(result['result']['items'][0]["geoLocation"]['lat'])
        if type(result['result']['items'][0]["geoLocation"]['long']) == list:
            long_ = float(result['result']['items'][0]["geoLocation"]['long'][0])
        else:
            long_ = float(result['result']['items'][0]["geoLocation"]['long'])
        row.append((lat,long_))
    
        uri = row[3]
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery("""
            SELECT ?place ?placeLabel ?lat ?long
             WHERE {
            <%s> geo:lat ?lat.
            <%s> geo:long ?long.
            }
            """ % (uri,uri))        
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for place in results['results']['bindings']:
            lat = float(place['lat']['value'])
            long_ = float(place['long']['value'])
            if vincenty(NICE_COORDINATES, (lat,long_)).km > 15:
                return False
        return True
    else:
        return False

In [23]:
with open('../data/dbpedia_match_nogeo.csv','r') as input_fp:
    reader=csv.reader(input_fp,)
    # skip header
    reader.next()
    rows = [ [unicode(col,'utf-8') for col in row] for row in reader]

In [24]:
for group in groups:
    best_score = max(group,key= lambda x: x[4])
    group = filter(lambda row: row[4] == best_score[4],group)
    group = filter(lambda row: row[4] > THRESHOLD, group)
    if len(group) >= 1:
        uri = group[0][3]
        
        # get dbpedia position
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery("""
            SELECT ?place ?placeLabel ?lat ?long
             WHERE {
            <%s> geo:lat ?lat.
            <%s> geo:long ?long.
            }
            """ % (uri,uri))        
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        if len(results['results']['bindings']) > 0:
            for place in results['results']['bindings']:
                lat = float(place['lat']['value'])
                long_ = float(place['long']['value'])
            # keep nearest to dbpedia poi
            min_distance = float('inf')
            min_poi = None
            for row in group:
                distance = vincenty((lat,long_), row[5]).km
                if distance < min_distance:
                    min_distance = distance
                    min_poi = row
            position = min_poi[5]
            min_poi[5] = position[0]
            min_poi.append(position[1])
            
        else:
            # keep nearest to center
            min_distance = float('inf')
            min_poi = None
            for row in group:
                distance = vincenty(NICE_COORDINATES, row[5]).km
                if distance < min_distance:
                    min_distance = distance
                    min_poi = row
            position = min_poi[5]
            min_poi[5] = position[0]
            min_poi.append(position[1])

        # get label
        
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery("""
                    SELECT ?label ?preferredLabel 
        WHERE { 
           <%s> rdfs:label ?label .
        FILTER (lang(?label) = "" || lang(?label) = "en") 
           OPTIONAL { 
             <%s> rdfs:label ?preferredLabel . 
             FILTER (lang(?preferredLabel) = "" || lang(?preferredLabel) = "fr") 
           } 
        }
                    """ % (min_poi[3],min_poi[3]))        
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for place in results['results']['bindings']:
            try:
                label = place['preferredLabel']['value']
            except KeyError:
                label = place['label']['value']
            min_poi[2] = label
        new_pois.append(min_poi)
        
new_pois = sorted(new_pois,key=lambda x: x[4],reverse=True)

In [25]:
new_pois

[[u'http://data.linkedevents.org/location/0e3c6deb-ce1b-3632-bc5f-9b72ec774416',
  u'Allianz Riviera',
  u'Allianz Riviera',
  u'http://dbpedia.org/resource/Allianz_Riviera',
  100.0,
  43.7327,
  7.18464],
 [u'http://data.linkedevents.org/location/998bc434-34ac-4ccd-aaf1-6d627a5b832c',
  u'Avenue Jean m\xe9decin',
  u'Avenue Jean-M\xe9decin',
  u'http://dbpedia.org/resource/Avenue_Jean_M\xe9decin',
  100.0,
  43.7006,
  7.26802],
 [u'http://data.linkedevents.org/location/759f64c3-8b08-499c-89a2-81db3d2e3b76',
  u'Observatoire de Nice',
  u'Observatoire de Nice',
  u'http://dbpedia.org/resource/Nice_Observatory',
  100.0,
  43.7274,
  7.29907],
 [u'http://data.linkedevents.org/location/7d6d0cad-d27e-365d-9751-236b3563baef',
  u'drap',
  u'Drap (Alpes-Maritimes)',
  u'http://dbpedia.org/resource/Drap',
  100.0,
  43.751,
  7.32169],
 [u'http://data.linkedevents.org/location/81d857b7-9195-4cb0-a0ad-127b9924ee92',
  u'Promenade des Anglais',
  u'Promenade des Anglais',
  u'http://dbpedia.

In [209]:
with open('data/dbpedia_match_nogeo_distinct.csv','wb') as output_fp:
    writer=csv.writer(output_fp,)
    # skip header
    writer.writerow(['3cixty_uri','3cixty_label','dbpedia_label','dbpedia_uri','score','latitude','longitude'])
    for row in new_pois:
        row_utf8 = [s.encode('utf-8') if type(s) == unicode else s for s in row]
        writer.writerow(row_utf8)

In [222]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
            SELECT ?label ?preferredLabel 
WHERE { 
   <http://dbpedia.org/resource/Nice> rdfs:label ?label .
FILTER (lang(?label) = "" || lang(?label) = "en") 
   OPTIONAL { 
     <http://dbpedia.org/resource/Nice> rdfs:label ?preferredLabel . 
     FILTER (lang(?preferredLabel) = "" || lang(?preferredLabel) = "tk") 
   } 
}
            """)        
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for place in results['results']['bindings']:
    try:
        label = place['preferredLabel']['value']
    except KeyError:
        label = place['label']['value']

Nice


In [233]:
with open('data/dbpedia_match_nogeo_distinct.csv','r') as input_fp:
    reader=csv.reader(input_fp,)
    # skip header
    reader.next()
    rows = [ [unicode(col,'utf-8') for col in row] for row in reader]

In [236]:
for row in rows:
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
    SELECT ?label ?preferredLabel 
    WHERE { 
       <%s> rdfs:label ?label .
    FILTER (lang(?label) = "" || lang(?label) = "en") 
       OPTIONAL { 
         <%s> rdfs:label ?preferredLabel . 
         FILTER (lang(?preferredLabel) = "" || lang(?preferredLabel) = "fr") 
       } 
    }
    """ % (row[3],row[3]))        
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    for place in results['results']['bindings']:
        try:
            label = place['preferredLabel']['value']
        except KeyError:
            label = place['label']['value']
    row[2] = label
with open('data/dbpedia_match_nogeo_distinct_convert.csv','wb') as output_fp:
    writer=csv.writer(output_fp,)
    # skip header
    writer.writerow(['3cixty_uri','3cixty_label','dbpedia_label','dbpedia_uri','score','latitude','longitude'])
    for row in rows:
        row_utf8 = [s.encode('utf-8') if type(s) == unicode else s for s in row]
        writer.writerow(row_utf8)

In [27]:
with open('../data/dbpedia_match_nogeo_distinct.csv','r') as input_fp:
    reader=csv.reader(input_fp,)
    # skip header
    reader.next()
    rows = [ [unicode(col,'utf-8') for col in row] for row in reader]

In [29]:
rows[0]

[u'http://data.linkedevents.org/location/0e3c6deb-ce1b-3632-bc5f-9b72ec774416',
 u'Allianz Riviera',
 u'Allianz Riviera',
 u'http://dbpedia.org/resource/Allianz_Riviera',
 u'100.0',
 u'43.7327',
 u'7.18464']

In [38]:
import geocoder
g = geocoder.google('Mountain View, CA')
g.latlng

for row in rows:
    g = geocoder.google(row[2])
    distance = distance = vincenty(g.latlng, (row[5],row[6])).meters
    print(row[2],distance)

(u'Allianz Riviera', 3137.1280827234914)
(u'Avenue Jean-M\xe9decin', 176.80810838880555)
(u'Observatoire de Nice', 2.5374551509731935)
(u'Drap (Alpes-Maritimes)', 380.22340836253153)
(u'Promenade des Anglais', 1632.6296287005166)
(u'Observatoire oc\xe9anologique de Villefranche-sur-Mer', 54.37585426368565)
(u'Palais Nikaia', 1.0782221554784936)
(u'Cimeti\xe8re du Ch\xe2teau (Nice)', 15.301449054410291)
(u'Parc Ph\u0153nix', 1884.8340890630282)
(u'Cimeti\xe8re russe de Nice', 12.10053439354003)
(u'Villefranche-sur-Mer', 51.615269421496116)
(u'H\xf4tel Negresco', 21.858847575751167)
(u'Cath\xe9drale Saint-Nicolas de Nice', 8.32827322578773)
(u'Terra Amata (site arch\xe9ologique)', 15.518097553314359)
(u'Villa Leopolda', 7.350323994216442)
(u'Saint-Laurent-du-Var', 619.3283230426936)
(u'Stade Charles-Ehrmann', 203.2407942664808)
(u'Jardin botanique de la ville de Nice', 51.69744710293782)
(u'Grand-H\xf4tel du Cap-Ferrat', 8005.706243791828)
(u"\xc9glise Sainte-Jeanne-d'Arc de Nice", 11.23

In [33]:
location

Location((43.705185, 7.19349289873, 0.0))